# Notes

* Need to work out pagination in queries, and how to recognize current data length such that only query new records 

In [1]:
# %pip install flipside

# Imports

In [6]:
from flipside import Flipside
from config import flipside_api_key, flipside_nft_holder_address

In [7]:
import pandas as pd
import os

In [8]:
sdk = Flipside(flipside_api_key)

## Data Checklist

```markdown
* [X] Curve Locks
* [X] Curve Votes

* [X] Gauge to LP map
* [X] Curve Liquidity

* [X] Convex Snapshot
* [X] Votium Bounties

* [X] StakeDAO Snapshot
```

# Variables

In [5]:
# title = "_may_12"

# Define Save Function

In [9]:
data_path = "/app/data/source"

def get_cwd():
    cwd_temp = os.getcwd()
    temp_split = cwd_temp.split('/')
    cwd = ""
    for x in temp_split:
        if x == 'experiments':
            break
        elif x == '':
            pass
        else:
            cwd += "/"+x       
    return cwd


def print_metrics(query):
    started_at = query.run_stats.started_at
    ended_at = query.run_stats.ended_at
    elapsed_seconds = query.run_stats.elapsed_seconds
    record_count = query.run_stats.record_count
    print(f"This query took ${elapsed_seconds} seconds to run and returned {record_count} records from the database.")

def query_and_save(_query, _filename, _page_size = 5000):
    # Initial Query
    print(f"___\n{_filename}")
    print(f"querying page: 1")
    query_result_set = sdk.query(_query, page_size = _page_size)
    df_output = pd.json_normalize(query_result_set.records)
  
    # Metrics
    print_metrics(query_result_set)
    
    # Handle Pagination
    if len(query_result_set.records) >= _page_size:
        i = 2
        keep_going = True
        while keep_going:
            print(f"querying page: {i}")
            extended_result_set = sdk.get_query_results(
                query_result_set.query_id,
                page_number=i,
                page_size=_page_size
            )
            # Metrics
            print_metrics(query_result_set)
            
            # Concat Dataframes
            df_local = pd.json_normalize(extended_result_set.records)
            df_output = pd.concat([df_output, df_local], ignore_index=True)
            
            # Check if continue
            print(len(extended_result_set.records) < _page_size)
            print(len(extended_result_set.records))
            if len(extended_result_set.records) < _page_size:
                keep_going = False
            i += 1
    # Save
    cwd = get_cwd()
    full_filename = cwd+ data_path + '/' + filename+'.csv'
    df_output.to_csv(full_filename) 
    
    return df_output

# Defining SQL

## Curve Locker

In [7]:
curve_locker_address = '0x5f3b5DfEb7B28CDbD7FAba78963EE202a494e2A2'

curve_locker_query = f"""SELECT 
  *,
  WEEK(BLOCK_TIMESTAMP) as WEEK_NUMBER,
  DAYOFWEEK(BLOCK_TIMESTAMP) as WEEK_DAY

FROM ethereum.core.ez_decoded_event_logs
WHERE CONTRACT_ADDRESS = lower('{curve_locker_address}')
"""
# AND BLOCK_TIMESTAMP <'2022-01-01 00:00:00.000'


In [8]:
filename = 'curve_locker'
df_curve_locker = query_and_save(curve_locker_query, filename)

___
curve_locker
querying page: 1
This query took $203 seconds to run and returned 99332 records from the database.
querying page: 2
This query took $203 seconds to run and returned 99332 records from the database.
False
5000
querying page: 3
This query took $203 seconds to run and returned 99332 records from the database.
False
5000
querying page: 4
This query took $203 seconds to run and returned 99332 records from the database.
False
5000
querying page: 5
This query took $203 seconds to run and returned 99332 records from the database.
False
5000
querying page: 6
This query took $203 seconds to run and returned 99332 records from the database.
False
5000
querying page: 7
This query took $203 seconds to run and returned 99332 records from the database.
False
5000
querying page: 8
This query took $203 seconds to run and returned 99332 records from the database.
False
5000
querying page: 9
This query took $203 seconds to run and returned 99332 records from the database.
False
5000
quer

## Curve Gauge Votes

In [9]:
curve_voter_address = '0x2F50D538606Fa9EDD2B11E2446BEb18C9D5846bB'

curve_gauge_vote_query = f"""SELECT 
    SYMBOL, 
    NAME, 
    WEEK(BLOCK_TIMESTAMP) as WEEK_NUMBER,
    DAYOFWEEK(BLOCK_TIMESTAMP) as WEEK_DAY,
    DECODED_LOG,
    TX_HASH,
    BLOCK_TIMESTAMP

FROM ethereum.core.ez_decoded_event_logs LOGS
 LEFT JOIN ethereum.core.dim_contracts CONTRACT
   ON CONTRACT.address = lower(LOGS.DECODED_LOG:gauge_addr::string)
WHERE CONTRACT_ADDRESS = lower('{curve_voter_address}')
AND EVENT_NAME = 'VoteForGauge'
ORDER BY BLOCK_TIMESTAMP ASC
"""

In [10]:
filename = 'curve_gauge_votes'
df_curve_locker = query_and_save(curve_gauge_vote_query, filename, 10000)

___
curve_gauge_votes
querying page: 1
This query took $56 seconds to run and returned 24460 records from the database.
querying page: 2
This query took $56 seconds to run and returned 24460 records from the database.
False
10000
querying page: 3
This query took $56 seconds to run and returned 24460 records from the database.
True
4460


## Snapshot Votes

In [11]:
convex_snapshot_votes_query = f"""
SELECT 
    PROPOSAL_ID, 
    PROPOSAL_START_TIME, 
    PROPOSAL_END_TIME, 
    PROPOSAL_TITLE, 
    PROPOSAL_AUTHOR,
    VOTE_OPTION,
    VOTING_POWER,
    VOTE_TIMESTAMP,
    QUORUM,
    CHOICES,
    VOTING_PERIOD,
    NETWORK,
    SPACE_ID,
    VOTER,
    ADDRESS_NAME,
    LABEL_TYPE, 
    LABEL_SUBTYPE,
    LABEL
FROM ethereum.core.ez_snapshot SNAPSHOT
 LEFT JOIN ethereum.core.dim_labels LABELS
   ON SNAPSHOT.VOTER = LABELS.ADDRESS
WHERE SPACE_ID = 'cvx.eth' 
AND PROPOSAL_TITLE LIKE 'Gauge Weight%'
AND VOTE_TIMESTAMP > '2022-12-20 00:00:00.000'
"""

In [12]:
stakedao_snapshot_votes_query = f"""
SELECT 
    PROPOSAL_ID, 
    PROPOSAL_START_TIME, 
    PROPOSAL_END_TIME, 
    PROPOSAL_TITLE, 
    PROPOSAL_AUTHOR,
    VOTE_OPTION,
    VOTING_POWER,
    VOTE_TIMESTAMP,
    QUORUM,
    CHOICES,
    VOTING_PERIOD,
    NETWORK,
    SPACE_ID,
    VOTER,
    ADDRESS_NAME,
    LABEL_TYPE, 
    LABEL_SUBTYPE,
    LABEL
FROM ethereum.core.ez_snapshot SNAPSHOT
 LEFT JOIN ethereum.core.dim_labels LABELS
   ON SNAPSHOT.VOTER = LABELS.ADDRESS
WHERE SPACE_ID = 'sdcrv.eth' 
AND PROPOSAL_TITLE LIKE 'Gauge vote - CRV%'
AND VOTE_TIMESTAMP > '2022-12-20 00:00:00.000'
"""

In [13]:
filename = 'convex_snapshot_votes'
df_convex_snapshot_votes = query_and_save(convex_snapshot_votes_query, filename, 1500)

___
convex_snapshot_votes
querying page: 1
This query took $77 seconds to run and returned 15746 records from the database.
querying page: 2
This query took $77 seconds to run and returned 15746 records from the database.
False
1500
querying page: 3
This query took $77 seconds to run and returned 15746 records from the database.
False
1500
querying page: 4
This query took $77 seconds to run and returned 15746 records from the database.
False
1500
querying page: 5
This query took $77 seconds to run and returned 15746 records from the database.
False
1500
querying page: 6
This query took $77 seconds to run and returned 15746 records from the database.
False
1500
querying page: 7
This query took $77 seconds to run and returned 15746 records from the database.
False
1500
querying page: 8
This query took $77 seconds to run and returned 15746 records from the database.
False
1500
querying page: 9
This query took $77 seconds to run and returned 15746 records from the database.
False
1500
quer

In [14]:
filename = 'stakedao_snapshot'
df_stakdao_snapshot_votes = query_and_save(stakedao_snapshot_votes_query, filename, 1500)

___
stakedao_snapshot
querying page: 1
This query took $11 seconds to run and returned 181 records from the database.


In [5]:
df_convex_snapshot_votes.vote_timestamp.max()

NameError: name 'df_curve_locker' is not defined

## Reference: Gauge to LP map

In [10]:
curve_voter_address = '0x2F50D538606Fa9EDD2B11E2446BEb18C9D5846bB'

gauge_to_lp_map_query = """
with v2_deployer as (
  SELECT
    BLOCK_TIMESTAMP as block_timestamp,
    DECODED_LOG:gauge::string as gauge_addr,
    DECODED_LOG:pool::string as pool_addr,
    DECODED_LOG:token::string as token_addr,
    'v2' as type
  
  
  FROM ethereum.core.ez_decoded_event_logs
  WHERE CONTRACT_ADDRESS = lower('0xF18056Bbd320E96A48e3Fbf8bC061322531aac99') -- v2 deployer
  AND EVENT_NAME = 'LiquidityGaugeDeployed'

),

factory_deployer as (
  SELECT 
    BLOCK_TIMESTAMP as block_timestamp,
    DECODED_LOG:gauge::string as gauge_addr,
    DECODED_LOG:pool::string as pool_addr,
    '' as token_addr,
    'factory' as type

  FROM ethereum.core.ez_decoded_event_logs as logs

  WHERE logs.CONTRACT_ADDRESS = lower('0xB9fC157394Af804a3578134A6585C0dc9cc990d4')  -- factory
  AND logs.EVENT_NAME = 'LiquidityGaugeDeployed'
),


combo as (
  SELECT gauge_addr, pool_addr, token_addr, type, block_timestamp FROM v2_deployer
  UNION
  SELECT gauge_addr, pool_addr, token_addr, type, block_timestamp  FROM factory_deployer
)

SELECT 
  combo.gauge_addr as GAUGE_ADDR, 
  combo.pool_addr as POOL_ADDR, 
  combo.token_addr as TOKEN_ADDR, 
  combo.type as TYPE, 
  combo.block_timestamp as BLOCK_TIMESTAMP,
  contracts.NAME as GAUGE_NAME,
  contracts.SYMBOL as GAUGE_SYMBOL,
  pool_contracts.NAME as POOL_NAME,
  pool_contracts.SYMBOL as POOL_SYMBOL,
  token_contracts.NAME as TOKEN_NAME,
  token_contracts.SYMBOL as TOKEN_SYMBOL
FROM combo
LEFT JOIN ethereum.core.dim_contracts as contracts
  ON combo.gauge_addr = contracts.ADDRESS
LEFT JOIN ethereum.core.dim_contracts as pool_contracts
  ON combo.pool_addr = pool_contracts.ADDRESS
LEFT JOIN ethereum.core.dim_contracts as token_contracts
  ON combo.token_addr = token_contracts.ADDRESS
"""

In [11]:
filename = 'gauge_to_lp_map'
df_curve_locker = query_and_save(gauge_to_lp_map_query, filename)

___
gauge_to_lp_map
querying page: 1
This query took $19 seconds to run and returned 272 records from the database.


# Votium Bounties 

In [12]:
votium_bounty_query = f"""SELECT 
  -- *
  EVENT_NAME,
  -- BLOCK_TIMESTAMP,
  TX_HASH,
  DECODED_LOG:_choiceIndex::int as choice_index,
  DECODED_LOG:_amount::int / pow(10,18) as amount,
  DECODED_LOG:_proposal::string as proposal_id,
  DECODED_LOG:_token::string as bounty_token_address,
  ORIGIN_FROM_ADDRESS,
  BLOCK_TIMESTAMP as block_timestamp,
  PRICE as price,
  price * amount as bounty_value,
  contracts.NAME as token_name,
  contracts.SYMBOL as token_symbol
  -- snap.CHOICES as CHOICES
FROM ethereum.core.ez_decoded_event_logs as logs
LEFT JOIN ethereum.core.fact_hourly_token_prices as prices
ON logs.DECODED_LOG:_token = prices.token_address
AND time_slice(logs.BLOCK_TIMESTAMP::timestamp_ntz, 1, 'HOUR') = prices.HOUR
LEFT JOIN ethereum.core.dim_contracts as contracts
ON logs.DECODED_LOG:_token = contracts.ADDRESS
-- time_slice(logs.BLOCK_TIMESTAMP::timestamp_ntz, 1, 'HOUR')

-- LEFT JOIN  ethereum.core.ez_snapshot as snap
-- on lower(logs.DECODED_LOG:_proposal) = lower(snap.PROPOSAL_ID)
WHERE CONTRACT_ADDRESS= lower('0x19bbc3463dd8d07f55438014b021fb457ebd4595')

AND EVENT_NAME = 'Bribed'
"""



In [13]:
filename = 'votium_bounty_for_round'
df_curve_locker = query_and_save(votium_bounty_query, filename)

___
votium_bounty_for_round
querying page: 1
This query took $15 seconds to run and returned 1726 records from the database.


# Curve Liquidity

In [ ]:
QueryRunExecutionError: QUERY_RUN_EXECUTION_ERROR: an error has occured while executing your query. errorName=OperationFailedError, errorMessage=SQL compilation error: error line 95 at position 74
invalid identifier 'PRICES.TOKEN_SYMBOL', errorData={
'code': '000904',
'data': {'age': 0, 'pos': -1, 'line': -1, 
'type': 'COMPILATION', 
'queryId': '01aea26f-0403-b9d3-3d4f-83014f15269b', 
'sqlState': '42000', 
'errorCode': '000904', 'internalError': False}, 
'name': 'OperationFailedError', 'message': "SQL compilation error: error line 95 at position 74
invalid identifier 'PRICES.TOKEN_SYMBOL'", 
'sqlState': '42000'

In [18]:
start_date = '2023-03-01 00:00:00.000'
curve_liquidity_query = f"""
with curve_swaps as (
SELECT
*
FROM ethereum.core.ez_dex_swaps
WHERE PLATFORM = 'curve' 
),

tokens_out as (
SELECT 
DISTINCT TOKEN_OUT as token,
CONTRACT_ADDRESS as pool_address,
POOL_NAME as pool_name,
SYMBOL_OUT as symbol
FROM curve_swaps
GROUP BY TOKEN_OUT, SYMBOL_OUT, CONTRACT_ADDRESS, POOL_NAME
),

tokens_in as (
SELECT 
DISTINCT TOKEN_IN as token,
CONTRACT_ADDRESS as pool_address,
POOL_NAME as pool_name,

SYMBOL_IN  as symbol
FROM curve_swaps
-- GROUP BY TOKEN_IN
),

pools as (
SELECT
DISTINCT CONTRACT_ADDRESS as pool_address,
POOL_NAME as pool_name,
'ETH' as symbol,
'' as token
FROM curve_swaps
-- GROUP BY CONTRACT_ADDRESS

),

tokens as (
select 
*
FROM tokens_out
UNION all
select 
*
FROM tokens_in
UNION all
select 
token, pool_address, pool_name, symbol
FROM pools
),

all_tokens as (
select 
* 
from tokens
group by token, symbol, pool_address, pool_name
),


-- COMBINE TOKENS WITH DATES
dates as (
select 
date_day 
from ethereum.core.dim_dates 
where date_day between '{start_date}' and current_date()
),

dates_x_tokens as (
select 
date_day,
symbol,
token,
pool_address,
pool_name

from dates
cross join all_tokens
ORDER BY date_day DESC
),


dates_x_tokens_x_price as (
select 
dates_x_tokens.date_day,
dates_x_tokens.symbol,
dates_x_tokens.token,
pool_address,
pool_name,
AVG(prices.PRICE) as daily_price
FROM dates_x_tokens
LEFT JOIN ethereum.core.fact_hourly_token_prices as prices
ON (dates_x_tokens.token = prices.TOKEN_ADDRESS 
  OR dates_x_tokens.token = prices.SYMBOL
  )
AND dates_x_tokens.date_day = prices.hour::date
GROUP BY (
  dates_x_tokens.date_day,
  dates_x_tokens.symbol,
  dates_x_tokens.token,
  dates_x_tokens.pool_address,
  dates_x_tokens.pool_name
  )
ORDER BY date_day DESC

),


balances as (
select

dates.date_day,
dates.pool_address,
dates.pool_name, 
dates.token,
dates.symbol,
sum(deltas.BAL_DELTA) as balance,
(balance * dates.daily_price) as balance_usd 

from dates_x_tokens_x_price as dates
left join ethereum.core.ez_balance_deltas as deltas
ON deltas.BLOCK_TIMESTAMP::date <= dates.date_day
AND deltas.USER_ADDRESS = dates.pool_address
AND (deltas.CONTRACT_ADDRESS = dates.token
OR dates.symbol = deltas.SYMBOL)
GROUP BY (
  dates.date_day,
  dates.pool_address,
  dates.pool_name, 
  dates.token,
  dates.symbol,
  dates.daily_price
  )
ORDER BY date_day DESC

) 


SELECT
  date_day, 
  pool_name,
  sum(balance) as current_bal,
  sum(balance_usd) as current_bal_usd,
  LISTAGG(symbol, ', ') as tradeable_assets,
  pool_address
FROM balances
GROUP BY date_day, pool_address, pool_name
ORDER BY date_day DESC

"""

In [19]:
filename = 'liquidity_general'
df_liquidity_general = query_and_save(curve_liquidity_query, filename)

___
liquidity_general
querying page: 1
